In [1]:
import os
import random
from diffusers import StableDiffusionPipeline
from PIL import Image, ImageDraw, ImageFont
from rembg import remove
import torch

/home/andresmtr/miniconda3/envs/AndTextVideo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [3]:
# Configurar el pipeline de Stable Diffusion
pipeline = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5").to("cuda")


Loading pipeline components...:  14%|█▍        | 1/7 [00:00<00:00,  6.94it/s]/home/andresmtr/miniconda3/envs/AndTextVideo/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00, 15.16it/s]


In [4]:
# Constantes para la miniatura
RESOLUTION = (1280, 720)  # Resolución estándar de miniatura de YouTube

In [5]:
# Función para eliminar el fondo de una imagen
def remove_background(image_path, output_path):
    with open(image_path, "rb") as inp_file:
        img_data = remove(inp_file.read())
    with open(output_path, "wb") as out_file:
        out_file.write(img_data)
    return output_path

In [6]:
# Función para seleccionar una imagen aleatoria de una carpeta
def get_random_image(folder_path):
    images = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
    return random.choice(images) if images else None

In [7]:
def get_unique_images(folder_path, num_images):
    images = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
    if len(images) < num_images:  # Si hay menos imágenes que las solicitadas
        return images  # Devuelve todas las disponibles
    return random.sample(images, num_images)

In [8]:
# Generar una imagen de fondo relacionada con el tema
def generate_background(prompt, output_path):
    # Establecer la resolución 1280x720 directamente en el pipeline
    image = pipeline(prompt, height=720, width=1280).images[0]
    image.save(output_path)
    return output_path

In [9]:
import random

def create_youtube_cover(background_path, my_photo_path, topic_images, title, output_path):
    # Cargar el fondo
    background = Image.open(background_path).convert("RGBA")
    width, height = RESOLUTION

    # Procesar y redimensionar la foto personal
    my_photo_no_bg = remove_background(my_photo_path, "temp_no_bg.png")
    my_photo = Image.open(my_photo_no_bg).convert("RGBA")

    # Ajustar la foto personal para que no se distorsione
    max_width = int(width * 0.4)  # Máximo 40% del ancho total
    max_height = height  # Máximo 100% de la altura
    aspect_ratio = my_photo.width / my_photo.height

    if my_photo.width > max_width or my_photo.height > max_height:
        if aspect_ratio > 1:  # Imagen apaisada
            new_width = max_width
            new_height = int(max_width / aspect_ratio)
        else:  # Imagen vertical
            new_height = max_height
            new_width = int(max_height * aspect_ratio)
        my_photo = my_photo.resize((new_width, new_height), Image.Resampling.LANCZOS)

    # Posicionar la foto personal en el lado derecho
    my_photo_position = (width - my_photo.width - 20, (height - my_photo.height) // 2)  # Lado derecho con margen de 20px
    background.paste(my_photo, my_photo_position, my_photo)

    # Ajustar la región ocupada por la foto personal
    photo_x_min = my_photo_position[0]
    photo_x_max = photo_x_min + my_photo.width
    photo_y_min = my_photo_position[1]
    photo_y_max = photo_y_min + my_photo.height

    # Añadir imágenes relacionadas al tema sin repetición
    topic_images = random.sample(topic_images, min(len(topic_images), 12))  # Máximo 12 imágenes únicas
    cols = 4  # Número máximo de columnas
    rows = (len(topic_images) // cols) + (1 if len(topic_images) % cols != 0 else 0)  # Calcular filas necesarias

    image_width = int(width / (cols + 1))  # Espacio disponible para cada imagen
    image_height = int(height / (rows + 1))

    for i, topic_image_path in enumerate(topic_images):
        # Quitar el fondo de la imagen relacionada
        topic_image_no_bg = remove_background(topic_image_path, f"temp_topic_{i}_no_bg.png")
        topic_image = Image.open(topic_image_no_bg).convert("RGBA")

        # Ajustar la imagen relacionada para que no se distorsione
        aspect_ratio = topic_image.width / topic_image.height

        if aspect_ratio > 1:  # Imagen apaisada
            new_width = image_width
            new_height = int(image_width / aspect_ratio)
        else:  # Imagen vertical
            new_height = image_height
            new_width = int(image_height * aspect_ratio)

        topic_image = topic_image.resize((new_width, new_height), Image.Resampling.LANCZOS)

        # Calcular la posición basada en la fila y columna
        row = i // cols
        col = i % cols
        x_offset = int((col + 1) * (width / (cols + 1)) - (new_width / 2))
        y_offset = int((row + 1) * (height / (rows + 1)) - (new_height / 2))

        # Asegurar que la imagen no se sobreponga con la foto personal
        if not (photo_x_min < x_offset + new_width < photo_x_max and photo_y_min < y_offset + new_height < photo_y_max):
            background.paste(topic_image, (x_offset, y_offset), topic_image)

    # Añadir el título con fondo negro
    draw = ImageDraw.Draw(background)
    font = ImageFont.truetype("Pemanka.ttf", size=50)

    # Obtener dimensiones del texto
    bbox = font.getbbox(title)
    text_width = bbox[2] - bbox[0]
    text_height = bbox[3] - bbox[1]

    # Posicionar el texto en el centro superior
    text_position = ((width - text_width) // 2, int(height * 0.05))

    # Calcular el área del rectángulo negro
    padding = 10  # Espacio adicional alrededor del texto
    rect_x0 = text_position[0] - padding
    rect_y0 = text_position[1] - padding
    rect_x1 = text_position[0] + text_width + padding
    rect_y1 = text_position[1] + text_height + padding

    # Dibujar el fondo negro
    draw.rectangle([rect_x0, rect_y0, rect_x1, rect_y1], fill="black")

    # Dibujar el texto en blanco
    draw.text(text_position, title, fill="white", font=font)

    # Guardar la miniatura final con la resolución 1280x720
    background.save(output_path)

In [10]:
input_promt = str(input('indique el promt: '))

In [11]:
background_prompt = input_promt
background_path = generate_background(background_prompt, "background.png")

100%|██████████| 50/50 [00:33<00:00,  1.50it/s]


In [12]:
# Seleccionar una foto personal aleatoria
my_photo_folder = "my_photos"  # Carpeta con tus fotos
my_photo_path = get_random_image(my_photo_folder)

In [13]:
# Seleccionar imágenes temáticas adicionales
topic_image_folder = "imagenes"  # Carpeta con imágenes relacionadas al tema
num_images_to_select = 5  # Número de imágenes que deseas seleccionar
topic_images = get_unique_images(topic_image_folder, num_images_to_select) 

In [14]:
input_title = str(input('indicar el titulo:'))

In [15]:
# Crear la portada
title = input_title
output_path = "youtube_cover_final.png"
create_youtube_cover(background_path, my_photo_path, topic_images, title, output_path)
print(f"Portada creada: {output_path}")

Portada creada: youtube_cover_final.png
